In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Data

In [2]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib') # Can't use !cd in co-lab
!./configure --prefix=/usr
!make
!make install
os.chdir('../')
!pip install TA-Lib

--2024-09-01 13:08:18--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-09-01 13:08:18--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-09-01 13:08:18--  http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1

In [3]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=493b937eb2a6fd84d0a4def6b81350639bed0b856f324e151ba3dc6432f9037a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
import tqdm
import yfinance as yf
from torch.utils.data import DataLoader
from torch.distributions import Categorical
import tqdm
from google.colab import runtime
# Finance Data
import pandas as pd
import pandas_ta as ta
import talib
from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from torch.utils.data import DataLoader


seaborn.set_context(context="talk")
%matplotlib inline

In [5]:
import pandas as pd
import numpy as np

def datetime_to_one_hot(datetime_values):
    # Convert input to a DatetimeIndex if it's not already
    if isinstance(datetime_values, pd.Series):
        datetime_values = pd.to_datetime(datetime_values)
    elif isinstance(datetime_values, pd.Index):
        datetime_values = pd.DatetimeIndex(datetime_values)
    else:
        raise ValueError("Input should be a Pandas Series or DatetimeIndex.")

    # Extract the day of the week (0=Monday, 6=Sunday)
    day_of_week = datetime_values.dayofweek

    # Extract the day of the month (1-31)
    day_of_month = datetime_values.day

    # Extract the week of the month (1-5)
    week_of_month = (datetime_values.day - 1) // 7 + 1

    # Extract the month of the year (1-12)
    month_of_year = datetime_values.month

    # One-hot encode each component
    one_hot_day_of_week = np.eye(7)[day_of_week]
    one_hot_day_of_month = np.eye(31)[day_of_month - 1]  # day_of_month ranges from 1-31, so subtract 1 for 0-based indexing
    one_hot_week_of_month = np.eye(5)[week_of_month - 1]  # week_of_month ranges from 1-5, so subtract 1 for 0-based indexing
    one_hot_month_of_year = np.eye(12)[month_of_year - 1]  # month_of_year ranges from 1-12, so subtract 1 for 0-based indexing

    # Concatenate all one-hot vectors along the second axis
    one_hot_combined = np.concatenate([one_hot_day_of_week, one_hot_day_of_month, one_hot_week_of_month, one_hot_month_of_year], axis=1)

    return one_hot_combined


In [6]:
def extract_data(start_year, end_year=2023, ticker="^SPX"):
    data = yf.Ticker(ticker).history(period="max")
    data = data.dropna()

    def add_features(data):
      # Assuming your DataFrame is named 'data'
      data.ta.sma(close="Close", length=50, append=True)
      data.ta.sma(close="Close", length=200, append=True)
      #data.ta.ichimoku(close="Close", append=True)
      #data.ta.macd(close="Close", append=True)
      data.ta.rsi(close="Close", append=True)
      data.ta.bbands(close="Close", append=True)
      data.ta.macd(close="Close", append=True)
      data.ta.ichimoku(close="Close", append=True)
      data.ta.smi(close="Close", append=True)
      data.ta.willr(close="Close", low="Low", high="High", append=True)
      data.ta.stoch(close="Close", low="Low", high="High", append=True)
      data.ta.fisher(low="Low", high="High", append=True)
      data.ta.atr(low="Low", high="High", close="Close", append=True)
      #data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
      data.ta.obv(volume="Volume", close="Close", append=True)
      data.ta.zscore(close="Close", append=True)
      data.ta.entropy(close="Close", append=True)
      return data

    data = add_features(data)
    ## Columns to Drop
    drop = ['Volume', 'Dividends', 'Stock Splits']
    data = data.drop(drop, axis=1)
    data = data.dropna()
    start_year = data.index[0].year if start_year is None else start_year
    data = data[data.index.year >= start_year-1] if start_year is not None else data
    data = data[data.index.year <= end_year] if end_year is not None else data
    #print(f"start year: {start_year}")
    #print(data)
    return data

df = extract_data(1990)
#df = df['Close']
df

,Open,High,Low,Close,SMA_50,SMA_200,RSI_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,...,SMIo_5_20_5,WILLR_14,STOCHk_14_3_3,STOCHd_14_3_3,FISHERT_9_1,FISHERTs_9_1,ATRr_14,OBV,ZS_30,ENTP_10
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,277.720001,277.720001,273.809998,275.309998,274.762200,268.111900,48.838065,274.613084,277.267993,279.922903,...,-0.057734,-77.409688,54.631409,58.669202,0.388937,0.797978,2.233492,2.680319e+10,0.233221,3.325722
1989-01-04 00:00:00-05:00,275.309998,279.750000,275.309998,279.429993,274.677599,268.165350,58.288271,274.697225,277.787994,280.878764,...,-0.011009,-15.361706,54.945716,57.522858,0.248036,0.388937,2.391100,2.695289e+10,1.217661,3.325502
1989-01-05 00:00:00-05:00,279.429993,281.510010,279.429993,280.010010,274.632200,268.221200,59.424554,274.949366,278.373999,281.798632,...,0.014795,-19.480489,62.582706,57.386610,0.507648,0.248036,2.368880,2.712693e+10,1.334578,3.326466
1989-01-06 00:00:00-05:00,280.010010,282.059998,280.010010,280.670013,274.598000,268.280000,60.735306,274.775049,278.628003,282.480957,...,0.030888,-16.848292,82.769838,66.766086,0.908969,0.507648,2.346102,2.728826e+10,1.478751,3.327660
1989-01-09 00:00:00-05:00,280.670013,281.890015,280.320007,280.980011,274.590000,268.368150,61.366579,275.167730,279.280005,283.392280,...,0.037330,-13.090746,83.526824,76.293122,1.352505,0.908969,2.290666,2.745144e+10,1.502972,3.329150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,4753.919922,4772.939941,4736.770020,4754.629883,4468.218584,4335.861046,71.069402,4694.534676,4741.732031,4788.929386,...,-0.054860,-10.098875,80.663261,83.908053,2.832599,2.986470,41.184150,1.379242e+12,1.666301,3.350524
2023-12-26 00:00:00-05:00,4758.859863,4784.720215,4758.450195,4774.750000,4477.157988,4340.426846,72.704374,4694.610700,4748.570020,4802.529339,...,-0.035086,-4.185293,90.737754,85.128169,2.967104,2.832599,40.391734,1.381756e+12,1.766583,3.350531
2023-12-27 00:00:00-05:00,4773.450195,4785.390137,4768.899902,4781.580078,4485.316992,4345.055946,73.256889,4692.545294,4751.212012,4809.878729,...,-0.019366,-1.730508,94.661774,88.687597,3.241961,2.967104,38.684484,1.384504e+12,1.750029,3.349759


In [7]:
from dataclasses import dataclass
@dataclass
class Args():
    simple_moving_average: bool = True
    rsi: bool = True
    bollinger_bands: bool = True
    macd: bool = True
    ichimoku: bool = True
    stochastic_momentum_index: bool = True
    stochastic_oscillator: bool = True
    williamR: bool = True
    fisherT: bool = True
    average_true_range: bool = True
    obv: bool = True
    zscore: bool = True
    entropy: bool = True
    candle_stick: bool = True
    relative_price: bool = True
    onehot_vector_size: int = 10

In [7]:
def extract_onehot_data(args, onehot_vector_size, start_year, end_year=2023, ticker="^SPX"):
    data = yf.Ticker(ticker).history(period="max")
    data = data[data["Volume"] != 0]
    data.ta.sma(close="Close", length=50, append=True)
    data.ta.sma(close="Close", length=200, append=True)
    #data.ta.ichimoku(close="Close", append=True)
    #data.ta.macd(close="Close", append=True)
    data.ta.rsi(close="Close", append=True)
    data.ta.bbands(close="Close", append=True)
    data.ta.macd(close="Close", append=True)
    data.ta.ichimoku(close="Close", append=True)
    data.ta.smi(close="Close", append=True)
    data.ta.willr(close="Close", low="Low", high="High", append=True)
    data.ta.stoch(close="Close", low="Low", high="High", append=True)
    data.ta.fisher(low="Low", high="High", append=True)
    data.ta.atr(low="Low", high="High", close="Close", append=True)
    data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
    data.ta.obv(volume="Volume", close="Close", append=True)
    data.ta.zscore(close="Close", append=True)
    data.ta.entropy(close="Close", append=True)
    candles = data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
    data = pd.concat([data, candles], axis=1)
    dates = data.index.to_list()

    cols = list(data.columns)
    cols_to_delete = cols[:-5]
    #print(cols)
    #print(cols_to_delete)

    # Relative Prices
    def map_to_index(x):
        # Negative changes: map to 0-(midpoint-1)
        bins = np.linspace(-0.1, 0.1, onehot_vector_size)
        idx = (np.abs(bins - x)).argmin()
        idx = np.array(idx)
        return idx

    if args.relative_price:
      data['Open/Close'] = data['Open'] / data['Close'] - 1.0
      data['High/Close'] = data['High'] / data['Close'] - 1.0
      data['Low/Close'] = data['Low'] / data['Close'] - 1.0
      data['Close_Change'] = data['Close'].pct_change()
      data['Open/Close'] = data['Open/Close'].round(3).map(map_to_index)
      data['High/Close'] = data['High/Close'].round(3).map(map_to_index)
      data['Low/Close'] = data['Low/Close'].round(3).map(map_to_index)
      data['Close_Change'] = data['Close_Change'].round(3).map(map_to_index)

    # SMA Crossover Strategy
    if args.simple_moving_average:
      data['golden_crossover'] = 0
      data['death_crossover'] = 0
      for i in range(1, len(dates)):
          if data.loc[dates[i], 'SMA_50'] > data.loc[dates[i], 'SMA_200'] and data.loc[dates[i-1], 'SMA_50'] <= data.loc[dates[i-1], 'SMA_200']:
              data.loc[dates[i], 'golden_crossover'] = 1
          elif data.loc[dates[i], 'SMA_50'] < data.loc[dates[i], 'SMA_200'] and data.loc[dates[i-1], 'SMA_50'] >= data.loc[dates[i-1], 'SMA_200']:
              data.loc[dates[i], 'death_crossover'] = 1

    # RSI
    if args.rsi:
      oversold = 30
      overbought = 70
      data['rsi_oversold'] = 0
      data['rsi_overbought'] = 0
      dates = data.index.to_list()
      for i in range(0, len(dates)):
        if data.loc[dates[i], 'RSI_14'] < oversold:
          data.loc[dates[i], 'rsi_oversold'] = 1
        if data.loc[dates[i], 'RSI_14'] > overbought:
          data.loc[dates[i], 'rsi_overbought'] = 1

    # Bollinger Bands Breakout Strategy
    if args.bollinger_bands:
      data['bbands_high_volatility'] = 0
      data['bbands_low_volatility'] = 0
      data['bbands_buy_signal'] = 0
      data['bbands_sell_signal'] = 0

      data['bbands_buy_signal'] = (data['Close'] > data['BBU_5_2.0']).astype(int)
      data['bbands_sell_signal'] = (data['Close'] < data['BBL_5_2.0']).astype(int)

    # MACD
    if args.macd:
      ## MACD Line Crosses Signal Line
      data['macd_golden_crossover'] = 0
      data['macd_death_crossover'] = 0
      for i in range(1, len(data)):
          if data.loc[dates[i], 'MACD_12_26_9'] > data.loc[dates[i], 'MACDs_12_26_9'] and data.loc[dates[i-1], 'MACD_12_26_9']<= data.loc[dates[i-1], 'MACDs_12_26_9']:
              data.loc[dates[i], 'macd_golden_crossover'] = 1
          elif data.loc[dates[i], 'MACD_12_26_9'] < data.loc[dates[i], 'MACDs_12_26_9'] and data.loc[dates[i-1], 'MACD_12_26_9'] >= data.loc[dates[i-1], 'MACDs_12_26_9']:
              data.loc[dates[i], 'macd_death_crossover'] = 1

      ## MACD histogram reversal
      data['macd_buy_signal_histogram'] = 0
      data['macd_sell_signal_histogram'] = 0
      for i in range(2, len(data)):
          if data.loc[dates[i], 'MACDh_12_26_9'] > data.loc[dates[i-1], 'MACDh_12_26_9'] and data.loc[dates[i-1], 'MACDh_12_26_9'] < data.loc[dates[i-2], 'MACDh_12_26_9']:
              data.loc[dates[i], 'macd_buy_signal_histogram'] = 1
          elif data.loc[dates[i], 'MACDh_12_26_9'] < data.loc[dates[i-1], 'MACDh_12_26_9'] and data.loc[dates[i-1], 'MACDh_12_26_9'] > data.loc[dates[i-2], 'MACDh_12_26_9']:
              data.loc[dates[i], 'macd_sell_signal_histogram'] = 1
      ## MACD Zero Line Cross
      data['macd_buy_signal_zero_cross'] = 0
      data['macd_sell_signal_zero_cross'] = 0
      for i in range(1, len(data)):
          if data.loc[dates[i], 'MACD_12_26_9'] > 0 and data.loc[dates[i-1], 'MACD_12_26_9'] <= 0:
              data.loc[dates[i], 'macd_buy_signal_zero_cross'] = 1
          elif data.loc[dates[i], 'MACD_12_26_9'] < 0 and data.loc[dates[i-1], 'MACD_12_26_9'] >= 0:
              data.loc[dates[i], 'macd_sell_signal_zero_cross'] = 1

    # Ichimoku
    if args.ichimoku:
      data['ichimoku_buy_signal'] = 0
      data['ichimoku_sell_signal'] = 0
      for i in range(26, len(data)):  # Starting from index 26 to ensure full visibility of Span B
          # Buy signal conditions
          if (data.loc[dates[i], 'Close'] > data.loc[dates[i], 'ISA_9'] and
              data.loc[dates[i], 'Close'] > data.loc[dates[i], 'ISB_26'] and
              data.loc[dates[i], 'ITS_9'] > data.loc[dates[i], 'IKS_26'] and
              data.loc[dates[i], 'ICS_26'] > data.loc[dates[i-26], 'Close']):
              data.loc[dates[i], 'ichimoku_buy_signal'] = 1
          # Sell signal conditions
          elif (data.loc[dates[i], 'Close'] < data.loc[dates[i], 'ISA_9'] and
                data.loc[dates[i], 'Close'] < data.loc[dates[i], 'ISB_26'] and
                data.loc[dates[i], 'ITS_9'] < data.loc[dates[i], 'IKS_26'] and
                data.loc[dates[i], 'ICS_26'] < data.loc[dates[i-26], 'Close']):
              data.loc[dates[i], 'ichimoku_sell_signal'] = 1

    # Stochastic Momentum Index (SMI)
    if args.stochastic_momentum_index:
      data['smi_buy_signal'] = 0
      data['smi_sell_signal'] = 0
      oversold_threshold = -40
      overbought_threshold = 40
      for i in range(1, len(data)):
          if (data.loc[dates[i], 'SMI_5_20_5'] > data.loc[dates[i], 'SMIs_5_20_5'] and
              data.loc[dates[i-1], 'SMI_5_20_5'] <= data.loc[dates[i-1], 'SMIs_5_20_5'] and
              data.loc[dates[i], 'SMI_5_20_5'] < oversold_threshold):
              data.loc[dates[i], 'smi_buy_signal'] = 1
          elif (data.loc[dates[i], 'SMI_5_20_5'] < data.loc[dates[i], 'SMIs_5_20_5'] and
                data.loc[dates[i-1], 'SMI_5_20_5'] >= data.loc[dates[i-1], 'SMIs_5_20_5'] and
                data.loc[dates[i], 'SMI_5_20_5'] > overbought_threshold):
              data.loc[dates[i], 'smi_sell_signal'] = 1

    # Stochastic Oscillator
    if args.stochastic_oscillator:
      data['stoch_osc_buy_signal'] = 0
      data['stoch_osc_sell_signal'] = 0
      oversold_threshold = 20
      overbought_threshold = 80
      for i in range(1, len(data)):
          if (data.loc[dates[i], 'STOCHk_14_3_3'] > data.loc[dates[i], 'STOCHd_14_3_3'] and
              data.loc[dates[i-1], 'STOCHk_14_3_3'] <= data.loc[dates[i-1], 'STOCHd_14_3_3'] and
              data.loc[dates[i], 'STOCHk_14_3_3'] < oversold_threshold):
              data.loc[dates[i], 'stoch_osc_buy_signal'] = 1
          elif (data.loc[dates[i], 'STOCHk_14_3_3'] < data.loc[dates[i], 'STOCHd_14_3_3'] and
                data.loc[dates[i-1], 'STOCHk_14_3_3'] >= data.loc[dates[i-1], 'STOCHd_14_3_3'] and
                data.loc[dates[i], 'STOCHk_14_3_3'] > overbought_threshold):
              data.loc[dates[i], 'stoch_osc_sell_signal'] = 1

    # Williams R
    if args.williamR:
      data['williamR_buy_signal'] = 0
      data['williamR_sell_signal'] = 0
      oversold_threshold = -80
      overbought_threshold = -20
      for i in range(1, len(data)):
          if (data.loc[dates[i], 'WILLR_14'] > oversold_threshold and
              data.loc[dates[i-1], 'WILLR_14'] <= oversold_threshold):
              data.loc[dates[i], 'williamR_buy_signal'] = 1
          elif (data.loc[dates[i], 'WILLR_14'] < overbought_threshold and
                data.loc[dates[i-1], 'WILLR_14'] >= overbought_threshold):
              data.loc[dates[i], 'williamR_sell_signal'] = 1

    # Fisher Transform
    if args.fisherT:
      data['fisherT_buy_signal'] = 0
      data['fisherT_sell_signal'] = 0
      for i in range(1, len(data)):
          if (data.loc[dates[i], 'FISHERT_9_1'] > data.loc[dates[i], 'FISHERTs_9_1'] and
              data.loc[dates[i-1], 'FISHERT_9_1'] <= data.loc[dates[i-1], 'FISHERTs_9_1']):
              data.loc[dates[i], 'fisherT_buy_signal'] = 1
          elif (data.loc[dates[i], 'FISHERT_9_1'] < data.loc[dates[i], 'FISHERTs_9_1'] and
                data.loc[dates[i-1], 'FISHERT_9_1'] >= data.loc[dates[i-1], 'FISHERTs_9_1']):
              data.loc[dates[i], 'fisherT_sell_signal'] = 1

    # Average True Range
    if args.average_true_range:
      atr_multiplier = 1.5
      data['atr_buy_signal'] = 0
      data['atr_sell_signal'] = 0
      for i in range(1, len(data)):
          if data.loc[dates[i], 'Close'] > data.loc[dates[i-1], 'Close'] + atr_multiplier * data.loc[dates[i], 'ATRr_14']:
              data.loc[dates[i], 'atr_buy_signal'] = 1
          elif data.loc[dates[i], 'Close'] < data.loc[dates[i-1], 'Close'] - atr_multiplier * data.loc[dates[i], 'ATRr_14']:
              data.loc[dates[i], 'atr_sell_signal'] = 1

    # OBV
    if args.obv:
      data['obv_buy_signal'] = 0
      data['obv_sell_signal'] = 0
      data['OBV_MA'] = data['OBV'].rolling(window=20).mean()
      for i in range(1, len(data)):
          if (data.loc[dates[i], 'OBV'] > data.loc[dates[i], 'OBV_MA'] and
              data.loc[dates[i-1], 'OBV'] <= data.loc[dates[i-1], 'OBV_MA']):
              data.loc[dates[i], 'obv_buy_signal'] = 1
          elif (data.loc[dates[i], 'OBV'] < data.loc[dates[i], 'OBV_MA'] and
                data.loc[dates[i-1], 'OBV'] >= data.loc[dates[i-1], 'OBV_MA']):
              data.loc[dates[i], 'obv_sell_signal'] = 1

    # Z Score
    if args.zscore:
      z_score_threshold = 1.5
      data['zscore_buy_signal'] = 0
      data['zscore_sell_signal'] = 0
      for i in range(1, len(data)):
          if data.loc[dates[i], 'ZS_30'] < -z_score_threshold:
              data.loc[dates[i], 'zscore_buy_signal'] = 1
          elif data.loc[dates[i], 'ZS_30'] > z_score_threshold:
              data.loc[dates[i], 'zscore_sell_signal'] = 1

    # Entropy
    if args.entropy:
      entropy_threshold = 0.7
      data['entropy_buy_signal'] = 0
      data['entropy_sell_signal'] = 0
      for i in range(1, len(data)):
          if data.loc[dates[i], 'ENTP_10'] < entropy_threshold:
              data.loc[dates[i], 'entropy_buy_signal'] = 1
          elif data.loc[dates[i], 'ENTP_10'] > entropy_threshold:
              data.loc[dates[i], 'entropy_sell_signal'] = 1

    data = data.dropna()
    drop = ['Volume', 'Dividends', 'Stock Splits']
    cols_to_delete.append('OBV_MA')
    data = data.drop(cols_to_delete, axis=1)
    print(list(data.columns))
    data = data.dropna()
    start_year = data.index[0].year if start_year is None else start_year
    data = data[data.index.year >= start_year-1] if start_year is not None else data
    data = data[data.index.year <= end_year] if end_year is not None else data
    #data.index = data.index.tz_convert(None)
    #data.index = data.index.strftime('%Y-%m-%d')
    #date_cols = ['date']
    #economic_data = pd.read_csv('/content/drive/MyDrive/code/fintransformer/economic_data.csv', index_col='date')
    #dfs = [data, economic_data]
    #merged = pd.concat(dfs, join='outer', axis=1)
    #merged = merged.dropna()
    return data

In [8]:
feature_args = Args()
onehot = extract_onehot_data(feature_args, onehot_vector_size=10, start_year=1990)
onehot

['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 'CDL_MORNINGSTAR', 'CDL_3BLACKCROWS', 'CDL_3LINESTRIKE', 'Open/Close', 'High/Close', 'Low/Close', 'Close_Change', 'golden_crossover', 'death_crossover', 'rsi_oversold', 'rsi_overbought', 'bbands_high_volatility', 'bbands_low_volatility', 'bbands_buy_signal', 'bbands_sell_signal', 'macd_golden_crossover', 'macd_death_crossover', 'macd_buy_signal_histogram', 'macd_sell_signal_histogram', 'macd_buy_signal_zero_cross', 'macd_sell_signal_zero_cross', 'ichimoku_buy_signal', 'ichimoku_sell_signal', 'smi_buy_signal', 'smi_sell_signal', 'stoch_osc_buy_signal', 'stoch_osc_sell_signal', 'williamR_buy_signal', 'williamR_sell_signal', 'fisherT_buy_signal', 'fisherT_sell_signal', 'atr_buy_signal', 'atr_sell_signal', 'obv_buy_signal', 'obv_sell_signal', 'zscore_buy_signal', 'zscore_sell_signal', 'entropy_buy_signal', 'entropy_sell_signal']


,CDL_EVENINGSTAR,CDL_3WHITESOLDIERS,CDL_MORNINGSTAR,CDL_3BLACKCROWS,CDL_3LINESTRIKE,Open/Close,High/Close,Low/Close,Close_Change,golden_crossover,...,fisherT_buy_signal,fisherT_sell_signal,atr_buy_signal,atr_sell_signal,obv_buy_signal,obv_sell_signal,zscore_buy_signal,zscore_sell_signal,entropy_buy_signal,entropy_sell_signal
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,5,5,4,4,0,...,0,1,0,0,0,0,0,0,0,1
1989-01-04 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,0,0,1,0,0,0,0,0,0,1
1989-01-05 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,1,0,0,0,1,0,0,0,0,1
1989-01-06 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,0,0,0,0,0,0,0,0,0,1
1989-01-09 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,0,0,0,0,0,0,0,1,0,1
2023-12-26 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,1,0,0,0,0,0,0,1,0,1
2023-12-27 00:00:00-05:00,0.0,0.0,0.0,0.0,0.0,4,5,4,5,0,...,0,0,0,0,0,0,0,1,0,1


In [138]:
data = onehot
data['year'] = data.index.year
data['date'] = data.index
target_cols = ['Open/Close',	'High/Close',	'Low/Close',	'Close_Change']
cols = list(data.columns)
for item in target_cols:
  cols.remove(item)
cols.remove('date')
cols.remove('year')
data = data[['date'] + target_cols + cols]

In [140]:
for i, idx in enumerate(data['Open/Close']):
    if i == 1:
      print(f"i: {i} | idx: {idx}")
      break

i: 1 | idx: 4


In [8]:
import pandas as pd
import numpy as np

def percentage_change_to_one_hot(df, vector_length=512):
    df = df.to_frame()
    # Define a function to map the percentage change to an index
    def map_to_index(x):
        # Negative changes: map to 0-(midpoint-1)
        bins = np.linspace(-0.1, 0.1, vector_length)
        idx = (np.abs(bins - x)).argmin()
        idx = np.array(idx)
        """print(x)
        print(np.abs(bins - x))
        print(bins)
        print(idx)
        one_hot = np.zeros((len(idx), self.bin_size))
        one_hot[np.arange(idx.size), idx] = 1"""
        return idx

    # Apply the mapping function to the DataFrame
    mapped_df = df.map(map_to_index)
    #print(mapped_df)

    # One-hot encode the mapped indices
    one_hot_encoded = np.eye(vector_length)[mapped_df.values]

    # Reshape to get the one-hot encoding for each column in a single array
    one_hot_encoded = one_hot_encoded.reshape(df.shape[0], -1)

    return one_hot_encoded


In [9]:
def map_to_index(x):
  # Negative changes: map to 0-(midpoint-1)
  bins = np.linspace(-0.1, 0.1, 10)
  idx = (np.abs(bins - x)).argmin()
  idx = np.array(idx)
  return idx

In [10]:
def add_onehot_embedding(df, column_name, vector_size):
  expanded_df = pd.DataFrame(np.zeros((df.shape[0], vector_size)), columns=[f'{column_name}_{i}' for i in range(vector_size)], index=df.index)
  for i, idx in enumerate(df[f'{column_name}']):
      expanded_df.iloc[i, idx] = 1

  result = pd.concat([df, expanded_df], axis=1)
  return result

In [ ]:
df = onehot
vector_size = 10
column_name = 'Open/Close'
print(type(df))
res = add_onehot_embedding(df,column_name, vector_size)

In [31]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class StockDataset(Dataset):
    def __init__(self, tickers, configs, flag='train', size=None, one_hot_datetime=False,
                 features='S', target='Close_Change', scale=True, timeenc=0, freq='d', batch_size=5,
                 data_start_year=1990, data_end_year=2023):
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]

        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.data_start_year = data_start_year
        self.data_end_year = data_end_year
        self.configs = configs
        self.features = features
        self.target = target
        self.scale = scale
        self.one_hot_datetime = one_hot_datetime
        self.timeenc = timeenc
        self.freq = freq
        self.scaler = StandardScaler()
        self.tickers = tickers.split()
        self.ticker_database = {}
        self.batch_size = batch_size
        self.min_year = 0
        self.__read_data__()
        self.target_cols = ['Open/Close',	'High/Close',	'Low/Close',	'Close_Change']

    def __len__(self):
        return len(self.years) - 1

    def get_min_year(self):
        min_year = 0
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_onehot_data(args=self.configs, onehot_vector_size=self.configs.onehot_vector_size, start_year=None, ticker=ticker)
            min_year = max(min_year, self.ticker_database[ticker].index.year.min())
        return min_year + 2

    def __read_data__(self):
        print(f"Loading following tickers: {self.tickers}\n")

        self.min_year = max(self.get_min_year(), self.data_start_year)
        print(f"Dataset Start Year: {self.min_year} | End Year: {self.data_end_year}")
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_onehot_data(args=self.configs, onehot_vector_size=self.configs.onehot_vector_size, start_year=self.min_year, end_year=self.data_end_year, ticker=ticker)
            self.ticker_database[ticker]['year'] = self.ticker_database[ticker].index.year

        self.years = self.ticker_database[self.tickers[0]]['year'].unique()
        print(f"years: {self.years}")
        self.data_by_year = {year: {ticker: self.ticker_database[ticker][self.ticker_database[ticker]['year'] == year] for ticker in self.tickers} for year in self.years}

        self.data_len = len(self.years)
        print(f"DateTime is one-hot: {self.one_hot_datetime}")

    def __getitem__(self, idx):
        """
        Final item form is a list containing [seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates]
        batches_x = batches[0]
        batches_y = batches[1]
        batches_x_mark = batches[2]
        batches_y_mark = batches[3]
        batches_x_dates = batches[4]
        batches_y_dates = batches[5]

        Each batches, contain batch data of size (batch_size, seq_len, num_features)
        For example, if seq_len = 24, batch_size = 5, num_features = 32,
        Each item of batches_x is a tensor of (5, 24, 32)
        """
        year = self.min_year + idx if idx >= 0 else self.max_year + idx + 1
        #print(f"Stock Dataset Yeaer: {year}")
        raw_datas = []

        for ticker in self.tickers:
            if year - 1 in self.data_by_year:
                prev_year_data = self.data_by_year[year - 1][ticker].tail(self.seq_len + self.pred_len - 1)
                #print(f"Prev year data for {ticker} in {year}: {prev_year_data.index}")
            else:
                print(f"Previous Year Data is Insufficient, Year: {year-1}, Ticker: {ticker}")
                prev_year_data = pd.DataFrame()

            ticker_data = pd.concat([prev_year_data, self.data_by_year[year][ticker]])
            #print(f"Ticker data for {ticker} in {year}: {ticker_data.index}")
            ticker_data['date'] = ticker_data.index
            raw_datas.append(ticker_data)
            """print(f"Ticker: {ticker}")
            print(len(ticker_data))
            print(ticker_data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []

        for item in raw_datas:
            x, y, x_mark = self.make_data(item)
            seq_x.append(x)
            seq_y.append(y)
            seq_x_mark.append(x_mark)
            #print(f"x shape: {x.shape} | y shape: {y.shape} | x_mark shape: {x_mark.shape}")

        # Combine all tickers into a single batch and slice them into mini-batches
        #print(len(seq_x))
        return self.create_batches(seq_x, seq_y, seq_x_mark), year

    def make_data(self, raw_data):
        cols = list(raw_data.columns)
        for item in self.target_cols:
          cols.remove(item)
        cols.remove('date')
        cols.remove('year')
        raw_data = raw_data[['date'] + self.target_cols + cols]


        if self.features == 'MS':
            cols_data = raw_data.columns[1:]
            data = raw_data[cols_data]

        elif self.features == 'S':
            data = raw_data['Close']

        ## Convert price data to one-hot vectors
        for target_col in self.target_cols:
          data = add_onehot_embedding(df=data, column_name=target_col, vector_size=self.configs.onehot_vector_size)

        data = data.drop(self.target_cols, axis=1)
        #print(type(pd.to_datetime(raw_data['date'].values)))

        ## Convert date to one-hot vectors
        data_stamp = datetime_to_one_hot(pd.to_datetime(raw_data['date'].values))

        """print("----Yearly data----")
        print(data[0].shape)
        print(len(data))
        print(data)"""
        seq_x = []
        seq_y = []
        seq_x_mark = []

        for i in range(len(data) - self.pred_len - self.seq_len + 1):
            x_begin = i
            x_end = x_begin + self.seq_len
            y_begin = x_end - 1
            y_end = x_end + self.pred_len

            seq_x.append(data[x_begin:x_end])
            seq_y.append(data[y_begin:y_end])
            seq_x_mark.append(data_stamp[x_begin:x_end])
            """print(f"data[s_begin:s_end]: {data[s_begin:s_end]}")
            print(f"data[r_begin:r_end]: {data[r_begin:r_end]}")
            print(f"raw_data['date'][s_begin:s_end]: {raw_data['date'][s_begin:s_end]}")
            print(f"raw_data['date'][r_begin:r_end]: {raw_data['date'][r_begin:r_end]}")
            print("=======================================")"""

        return torch.tensor(np.array(seq_x)), torch.tensor(np.array(seq_y)), torch.tensor(np.array(seq_x_mark))

    def create_batches(self, seq_x, seq_y, seq_x_mark):
        batches_x = []
        batches_y = []
        batches_x_mark = []
        batch_size = self.batch_size
        for x, y, x_mark in zip(seq_x, seq_y, seq_x_mark):
            for i in range(0, x.shape[0], batch_size):
                batches_x.append(x[i:i + batch_size])
                batches_y.append(y[i:i + batch_size])
                batches_x_mark.append(x_mark[i:i + batch_size])

        return batches_x, batches_y, batches_x_mark

In [22]:
dataset = StockDataset(configs=feature_args, tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS')

Loading following tickers: ['^SPX']

['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 'CDL_MORNINGSTAR', 'CDL_3BLACKCROWS', 'CDL_3LINESTRIKE', 'Open/Close', 'High/Close', 'Low/Close', 'Close_Change', 'golden_crossover', 'death_crossover', 'rsi_oversold', 'rsi_overbought', 'bbands_high_volatility', 'bbands_low_volatility', 'bbands_buy_signal', 'bbands_sell_signal', 'macd_golden_crossover', 'macd_death_crossover', 'macd_buy_signal_histogram', 'macd_sell_signal_histogram', 'macd_buy_signal_zero_cross', 'macd_sell_signal_zero_cross', 'ichimoku_buy_signal', 'ichimoku_sell_signal', 'smi_buy_signal', 'smi_sell_signal', 'stoch_osc_buy_signal', 'stoch_osc_sell_signal', 'williamR_buy_signal', 'williamR_sell_signal', 'fisherT_buy_signal', 'fisherT_sell_signal', 'atr_buy_signal', 'atr_sell_signal', 'obv_buy_signal', 'obv_sell_signal', 'zscore_buy_signal', 'zscore_sell_signal', 'entropy_buy_signal', 'entropy_sell_signal']
Dataset Start Year: 1990 | End Year: 2023
['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 

# Encoder Model

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [13]:
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [14]:
class Encoder(nn.Module):
  def __init__(self, layer, N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self, x, mask=None):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [15]:
class LayerNorm(nn.Module):
  def __init__(self, features, eps=1e-6):
    super(LayerNorm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2 * (x-mean) / (std+self.eps) + self.b_2

In [16]:
class SublayerConnection(nn.Module):
  def __init__(self, size, dropout):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [17]:
class EncoderLayer(nn.Module):
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 2)
    self.size = size

  def forward(self, x, mask):
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
    return self.sublayer[1](x, self.feed_forward)

In [18]:
def attention(query, key, value, mask=None, dropout=None):
  d_k = query.size(-1)
  scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = F.softmax(scores, dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return torch.matmul(p_attn, value), p_attn

In [19]:
class MultiHeadedAttention(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(MultiHeadedAttention, self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout= nn.Dropout(p=dropout)

  def forward(self, query, key, value, mask=None):
    if mask is not None:
      mask = mask.unsqueeze(1)
    nbatches = query.size(0)
    query, key, value = [linear(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
                          for linear, x in zip(self.linears, (query,key,value))]
    x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
    #print(f"before concat: {x.shape}")
    x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h * self.d_k)
    #print(f"after concat: {x.shape}")
    return self.linears[-1](x)

In [20]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PositionwiseFeedForward, self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [21]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [22]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        #print(f"pe shape: {self.pe.shape}")
        #print(f"x shape: {x.shape}")
        x = x + Variable(self.pe[:, :x.size(1)],
                         requires_grad=False)
        return self.dropout(x)

In [23]:
def make_encoder(N=6, d_model=256, d_ff=512, h=8, dropout=0.1):
  c = copy.deepcopy
  attn = MultiHeadedAttention(h, d_model)
  ff = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)
  #embedding = Embeddings(d_model, src_vocab)
  model = c(Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N))
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)
  return model

In [24]:
class Model(nn.Module):
  def __init__(self, configs):
    super(Model, self).__init__()
    self.encoder = make_encoder(N=configs.N, d_model=configs.d_model, d_ff=configs.d_ff, h=configs.n_heads, dropout=configs.dropout)
    self.time_embedding = nn.Linear(configs.datetime_features, configs.d_model)
    self.feature_embedding = nn.Linear(configs.feature_num, configs.d_model)
    self.classification = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=0),
            nn.Tanh(),
            nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=0),
            nn.Tanh(),
            nn.Conv1d(in_channels=128, out_channels=1, kernel_size=1, stride=1, padding=0),
            nn.Tanh(),
            nn.Flatten(),
            nn.Linear(26, configs.num_classes)
    )
    self.simple_classifier = nn.Sequential(
        nn.Linear(512, 64),
        nn.GELU(),
        nn.Linear(64, configs.num_classes),
    )

  def forward(self, x, x_mark_enc):
    if torch.is_tensor(x) is not True:
      x = torch.Tensor(x)
    x = self.feature_embedding(x) + self.time_embedding(x_mark_enc)
    x = self.encoder(x)
    #x = x.permute(0,2,1)
    #print(f"encoder out shape: {x.shape}")
    #print(f"x shape: {x[:,-1,:].shape}")
    x = self.simple_classifier(x)[:,-1,:]
    return x

In [25]:
import torch

def classification_evaluate_direction(output, batch_y):
    batch_size = output.size(0)

    prediction_direction = output.argmax(axis=1)
    correct_direction = batch_y.argmax(axis=1)


    correct_predictions = (prediction_direction == correct_direction).sum().item()
    #print(f"correct_predictions: {correct_predictions}")
    total_comparisons = batch_size
    #print(f"total_comparisons: {total_comparisons}")
    accuracy = correct_predictions / total_comparisons

    # Compute baseline predictions (always predict increase)
    baseline_direction = torch.zeros_like(correct_direction)  # Baseline always predicts increase
    baseline_correct = (baseline_direction == correct_direction).sum().item()
    baseline_accuracy = baseline_correct / total_comparisons

    """print(f"prediction_direction: {prediction_direction}")
    print(f"correct_direction: {correct_direction}")
    print(f"baseline_direction: {baseline_direction}")
    print(f"Accuracy: {accuracy:.4f} | Baseline Accuracy: {baseline_accuracy:.4f}")"""

    return correct_predictions, baseline_correct, total_comparisons

In [26]:
def compute_direction(tensor):
  batch_size = tensor.size(0)
  comparison = tensor[:, 1] > tensor[:, 0]
  # Convert the comparison result to a one-hot vector
  one_hot = torch.zeros(batch_size, 2)
  one_hot[comparison, 0] = 1  # [1, 0] when the last value is greater
  one_hot[~comparison, 1] = 1  # [0, 1] when the first value is greater
  return one_hot

tensor = torch.randn(5, 2)
print(tensor)
compute_direction(tensor)

tensor([[-1.2174, -0.2014],
        [-0.8901,  0.8258],
        [ 0.6263,  1.0172],
        [-1.3590, -0.8345],
        [ 0.5684, -3.4330]])


tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])

# Test

In [47]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    n_heads: int = 8
    d_model: int = 512
    d_ff: int = 2048
    N: int = 6
    feature_num: int = 2085
    learning_rate: float = 0.0001
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    num_classes: int = 2

    ## Data
    simple_moving_average: bool = True
    rsi: bool = True
    bollinger_bands: bool = True
    macd: bool = True
    ichimoku: bool = True
    stochastic_momentum_index: bool = True
    stochastic_oscillator: bool = True
    williamR: bool = True
    fisherT: bool = True
    average_true_range: bool = True
    obv: bool = True
    zscore: bool = True
    entropy: bool = True
    candle_stick: bool = True
    relative_price: bool = True
    onehot_vector_size: int = 512

    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = False
    datetime_features: int = 55

    ## Training
    run_name: str = "aapl_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "aapl"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "transformer_rolling"

In [48]:
configs = Args()
transformers = Model(configs)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_optim = torch.optim.Adam(transformers.parameters(), lr=configs.learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
dataset = StockDataset(configs=configs, tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS', batch_size=5, one_hot_datetime=True)

Loading following tickers: ['^SPX']

['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 'CDL_MORNINGSTAR', 'CDL_3BLACKCROWS', 'CDL_3LINESTRIKE', 'Open/Close', 'High/Close', 'Low/Close', 'Close_Change', 'golden_crossover', 'death_crossover', 'rsi_oversold', 'rsi_overbought', 'bbands_high_volatility', 'bbands_low_volatility', 'bbands_buy_signal', 'bbands_sell_signal', 'macd_golden_crossover', 'macd_death_crossover', 'macd_buy_signal_histogram', 'macd_sell_signal_histogram', 'macd_buy_signal_zero_cross', 'macd_sell_signal_zero_cross', 'ichimoku_buy_signal', 'ichimoku_sell_signal', 'smi_buy_signal', 'smi_sell_signal', 'stoch_osc_buy_signal', 'stoch_osc_sell_signal', 'williamR_buy_signal', 'williamR_sell_signal', 'fisherT_buy_signal', 'fisherT_sell_signal', 'atr_buy_signal', 'atr_sell_signal', 'obv_buy_signal', 'obv_sell_signal', 'zscore_buy_signal', 'zscore_sell_signal', 'entropy_buy_signal', 'entropy_sell_signal']
Dataset Start Year: 1990 | End Year: 2023
['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
range_limit = configs.rolling_window+configs.validation_years+configs.test_years
loop_range = tqdm.tqdm(range(len(dataset)-range_limit))
transformers = transformers.to(device)
for iteration in loop_range:
  for window_iteration in range(configs.window_epoch):
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    starting_idx = iteration // 10
    for i in range((configs.rolling_window+configs.validation_years+configs.test_years)):
        validation, test = False, False
        if i == configs.rolling_window:
          validation = True
        elif i == (configs.rolling_window+1):
          test = True
        else:
          batches, year  = dataset[i+iteration]
          batches_x, batches_y, batches_x_mark = batches[0], batches[1], batches[2]
          for i, (batch_x, batch_y, batch_x_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark)):
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            #print(batch_x.shape)

            outputs = transformers(batch_x, batch_x_mark)
            #print(f"outputs: {outputs}")

            f_dim = -configs.onehot_vector_size if configs.features == 'MS' else 0

            #print(f"last 2 batch_y: {batch_y[:, -(configs.pred_len+1):, f_dim:].argmax(axis=2)}")
            batch_y_direction = compute_direction(batch_y[:, -(configs.pred_len+1):, f_dim:].argmax(axis=2)).to(device)
            #print(f"batch_y_direction: {batch_y_direction}")
            loss = loss_fn(outputs, batch_y_direction)
            epoch_loss.append(loss.item())
            loss.backward()
            model_optim.step()

            #print(f"outputs shape: {outputs.shape} | true_batch_y[:,-97:,:]: {true_batch_y[:,-97:,-1:].shape}")
            correct_predictions, baseline_correct, total_comparisons = classification_evaluate_direction(outputs, batch_y_direction)
            total_hits += correct_predictions
            total_data +=total_comparisons
            total_ol += baseline_correct
            break
        print(f"Year: {year} total_comparisons: {total_data} training_hit_ratio: {total_hits/total_data} | ol_hit_ratio: {total_ol/total_data}")
        break
    break
  break

  0%|          | 0/21 [00:00<?, ?it/s]

x shape: torch.Size([253, 36, 2085]) | y shape: torch.Size([253, 2, 2085]) | x_mark shape: torch.Size([253, 36, 55])


  0%|          | 0/21 [00:00<?, ?it/s]

Year: 1990 total_comparisons: 5 training_hit_ratio: 0.8 | ol_hit_ratio: 0.4


# Trainer

In [37]:
class Trainer():
  def __init__(self, configs):
    self.configs = configs

    ticker_str = configs.tickers.replace(" ", "_")
    self.run_name = f"[{ticker_str}]_valYrs:{self.configs.validation_years}_testYrs{self.configs.test_years}_reset:{self.configs.reset_model}_{configs.run_name}"
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.set_seed(configs.seed)
    self.dataset = self.make_data()
    self.model = Model(configs)
    self.model.to(self.device)
    self.loss_fn = nn.BCEWithLogitsLoss()
    self.model_optim = torch.optim.Adam(self.model.parameters(), lr=configs.learning_rate)
    self.writer = SummaryWriter(f"/content/drive/MyDrive/code/fintransformer/runs/{self.run_name}")
    path = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}"
    if not os.path.exists(path):
      os.mkdir(path)
      print(f"Save File Directory Made at {path}\n")
    else:
      print(f"Directory Already Exists at {path}")

  def set_seed(self, seed):
      torch.manual_seed(seed)
      torch.cuda.manual_seed(seed)
      #torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
      np.random.seed(seed)
      random.seed(seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

  def make_data(self):
    def collate_fn(batch):
      seq_x, seq_y, seq_x_mark, seq_y_mark, year = batch[0]
      return seq_x, seq_y, seq_x_mark, seq_y_mark, year
    dataset = StockDataset(configs=self.configs, tickers=self.configs.tickers, timeenc=1, freq='d', size=[self.configs.seq_len, self.configs.label_len, self.configs.pred_len],
                           features=self.configs.features, batch_size=self.configs.batch_size, one_hot_datetime=self.configs.one_hot_datetime,
                           data_start_year=self.configs.data_start_year, data_end_year=self.configs.data_end_year)
    """prices, labels, year = next(iter(val_loader))
    feature_num = prices.shape[2]
    label_num = labels.shape[1]
    print(f"Data shape: {prices.shape} | Num Features: {feature_num}")"""
    return dataset

  def run(self):
    ### Training ###
    print(f"Training model: {self.configs.run_name}")
    print()
    range_limit = self.configs.rolling_window + self.configs.validation_years + self.configs.test_years
    loop_range = tqdm.tqdm(range(len(self.dataset)-range_limit))
    recent_save_path = ""
    val_acc_arr = []
    test_acc_arr = []
    for iteration in loop_range:
      highest_acc = 0.0
      highest_test_acc = 0.0
      if self.configs.reset_model and iteration != 0:
        self.model = Model(self.configs)
        self.model.to(self.device)
        self.model_optim = torch.optim.Adam(self.model.parameters(), lr=self.configs.learning_rate)
        print(f"Model Has Been Reset to Random Parameters")
      else:
        self.load_model(recent_save_path) if recent_save_path != "" else None
      for window_iteration in range(self.configs.window_epoch):
        train_loss, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year = self.train(iteration, window_iteration, highest_acc)
        if average_val_accuracy > highest_acc:
            highest_acc = average_val_accuracy
            highest_test_acc = average_test_accuracy
            recent_save_path = self.save_model(average_val_accuracy, average_test_accuracy, window_iteration, validation_year, test_year)
      #self.writer.add_scalar("Train/Test Loss", train_loss, iteration)
      val_acc_arr.append([highest_acc, validation_year])
      test_acc_arr.append([highest_test_acc, test_year])
      print(f"***********************************************************")
      print(f"Highest Validation Accuracy in {validation_year[0]}~{validation_year[-1]}: {highest_acc*100:.2f}% | Highest Test Accuracy in {test_year}: {average_test_accuracy*100:.2f}%")
      print(f"***********************************************************")
      print("=============================New Training Set====================================")
    self.writer.flush()
    self.writer.close()
    #print(f"Highest Accuracy in Validation Set: {highest_acc*100:.2f}")
    print()
    for val, test in zip(val_acc_arr, test_acc_arr):
      print(f"Validation Accuracy in {val[1][0]}~{val[1][-1]} : {val[0]*100:.2f}% | Test Accuracy in {test[1]}: {test[0]*100:.2f}%")

    average_first_elements_val = sum(item[0] for item in val_acc_arr) / len(val_acc_arr)
    average_first_elements_test = sum(item[0] for item in test_acc_arr) / len(test_acc_arr)
    print(f"Average Validation Accuracy: {average_first_elements_val*100:.2f}% | Average Test Accuracy: {average_first_elements_test*100:.2f}%")
    return

  def train(self, iteration, window_iteration, highest_acc):
    epoch_loss = []
    val_epoch_loss = []
    training_start_year, training_end_year = 0, 0
    training_total_hits = 0
    training_total_data = 0
    training_total_ol = 0
    validation_year, test_year = [], 0
    validation_total_hits = 0
    validation_total_data = 0
    validation_total_ol = 0
    test_total_hits = 0
    test_total_data = 0
    test_total_ol = 0
    for i in range((self.configs.rolling_window+self.configs.validation_years+self.configs.test_years)):
      validation, test = False, False
      if i >= self.configs.rolling_window and i <= (self.configs.rolling_window+self.configs.validation_years-1):
        batches, year = self.dataset[i+iteration]
        #print(f"validation year: {year}")
        batches_x, batches_y, batches_x_mark = batches[0], batches[1], batches[2]
        accuracy, only_long, val_loss_epoch, total_hits, total_data, total_ol = self.eval_once(batches_x, batches_y, batches_x_mark, highest_acc=0, train_acc=0)
        validation_total_hits += total_hits
        validation_total_data += total_data
        validation_total_ol += total_ol
        val_epoch_loss.append(val_loss_epoch)
        validation_year.append(year)
        validation = True
        print(f"Year: {year} | Validation Accuracy: {(total_hits/total_data)*100:.2f}% | Only Long Accuracy: {(total_ol/total_data)*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
      elif i >= (self.configs.rolling_window+self.configs.validation_years):
        batches, year = self.dataset[i+iteration]
        #print(f"test year: {year}")
        batches_x, batches_y, batches_x_mark = batches[0], batches[1], batches[2]
        accuracy, only_long,  total_hits, total_data, total_ol = self.test(batches_x, batches_y, batches_x_mark)
        test_total_hits += total_hits
        test_total_data += total_data
        test_total_ol += total_ol
        test_year = year
        test = True
      else:
        self.model.train()
        batches, year = self.dataset[i+iteration]
        training_start_year = year if i == 0 else training_start_year
        training_end_year = year if i == self.configs.rolling_window-1 else training_end_year
        #print(f"train year: {year}")
        batches_x, batches_y, batches_x_mark = batches[0], batches[1], batches[2]
        for j, (batch_x, batch_y, batch_x_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark)):
          self.model_optim.zero_grad()
          batch_x = batch_x.float().to(self.device)
          batch_y = batch_y.float().to(self.device)
          batch_x_mark = batch_x_mark.float().to(self.device)
          #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
          #print(batch_x.shape)

          outputs = self.model(batch_x, batch_x_mark)

          f_dim = -self.configs.onehot_vector_size if self.configs.features == 'MS' else 0
          #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
          batch_y_direction = compute_direction(batch_y[:, -(self.configs.pred_len+1):, f_dim:].argmax(axis=2)).to(self.device)
          #print(f"outputs: {outputs.shape} | batch_y_direction: {batch_y_direction.shape}")
          loss = self.loss_fn(outputs, batch_y_direction)
          epoch_loss.append(loss.item())
          loss.backward()
          self.model_optim.step()
        #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

          hit_count, only_long, count = classification_evaluate_direction(outputs, batch_y_direction)
          training_total_hits += hit_count
          training_total_data += count
          training_total_ol += only_long

    loss_epoch = np.mean(epoch_loss)
    average_val_accuracy = validation_total_hits / validation_total_data
    average_val_only_long = validation_total_ol / validation_total_data
    average_val_loss = np.mean(val_epoch_loss)
    average_test_accuracy = test_total_hits / test_total_data
    average_test_only_long = test_total_ol / test_total_data
    train_accuracy = training_total_hits / training_total_data
    train_only_long = training_total_ol / training_total_data
    print(f"Training Years: {training_start_year}~{training_end_year} | Training Accuracy: {train_accuracy*100:.2f}% | Training OL Accuracy: {train_only_long*100:.2f}%")
    print(f"Year: {validation_year[0]}~{validation_year[-1]} Validation Accuracy: {average_val_accuracy*100:.2f}% | Only Long Accuracy: {average_val_only_long*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
    print(f"Year: {test_year} | Test Accuracy: {average_test_accuracy*100:.2f}% | Only Long Accuracy: {average_test_only_long*100:.2f}%")
    print(f"------------------------------------------------")
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Accuracy", train_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Loss", loss_epoch, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Loss", average_val_loss, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Accuracy", average_val_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Test Accuracy", average_test_accuracy, window_iteration)
      #print(f"Year: {year} training_hit_ratio: {hit_count/batch_x.shape[0]} | ol_hit_ratio: {only_long/batch_x.shape[0]}")

    #print(f"train accuracy: {accuracy*100:.2f}% | only_long: {only_long*100:.2f}%")
    return loss_epoch, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year

  def eval_once(self, batches_x, batches_y, batches_x_mark, highest_acc, train_acc, last=False):
    self.model.eval()
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        #print(batch_x.shape)

        outputs = self.model(batch_x, batch_x_mark)

        f_dim = -self.configs.onehot_vector_size if self.configs.features == 'MS' else 0
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
        batch_y_direction = compute_direction(batch_y[:, -(self.configs.pred_len+1):, f_dim:].argmax(axis=2)).to(self.device)

        loss = self.loss_fn(outputs, batch_y_direction)
        epoch_loss.append(loss.item())

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        hit_count, only_long, count = classification_evaluate_direction(outputs, batch_y_direction)
        total_hits += hit_count
        total_data += count
        total_ol += only_long
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    loss_epoch = np.mean(epoch_loss)
    #print(f"validation accuracy: {accuracy*100:.2f}% | val highest_acc: {highest_acc*100:.2f}% | val only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, loss_epoch, total_hits, total_data, total_ol

  def test(self, batches_x, batches_y, batches_x_mark):
    self.model.eval()
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        #print(batch_x.shape)

        outputs = self.model(batch_x, batch_x_mark)

        f_dim = -self.configs.onehot_vector_size if self.configs.features == 'MS' else 0
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
        batch_y_direction = compute_direction(batch_y[:, -(self.configs.pred_len+1):, f_dim:].argmax(axis=2)).to(self.device)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        hit_count, only_long, count = classification_evaluate_direction(outputs, batch_y_direction)
        total_hits += hit_count
        total_data += count
        total_ol += only_long
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    #print(f"test accuracy: {accuracy*100:.2f}% | test only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, total_hits, total_data, total_ol

  def save_model(self, val_acc, test_acc, epoch, validation_year, test_year):
    PATH = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}/reset:{self.configs.reset_model}_valYear:{validation_year}_testYear:{test_year}.pt"
    torch.save({
            'model_state_dict': self.model.state_dict(),
            'model_optimizer_state_dict': self.model_optim.state_dict(),
            'val_acc': val_acc,
            'validation_year': validation_year,
            'test_acc': test_acc,
            'test_year': test_year,
            'configs': self.configs}, PATH)
    print(f"Model Saved at {PATH}")
    return PATH

  def load_model(self, path):
    checkpoint = torch.load(path, map_location=torch.device(self.device))
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.model_optim.load_state_dict(checkpoint['model_optimizer_state_dict'])
    print(f"Model loaded from {path}")
    print(f"Model Validation Accuracy: {checkpoint['val_acc']*100:.2f}% | Test Accuracy: {checkpoint['test_acc']*100:.2f}%")
    """try:
      self.configs = checkpoint['configs']
    except:
      print("No Configs Found in torch file")"""
    return

In [38]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    n_heads: int = 8
    d_model: int = 512
    d_ff: int = 2048
    N: int = 6
    feature_num: int = 2085
    learning_rate: float = 0.0001
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    num_classes: int = 2

    ## Data
    simple_moving_average: bool = True
    rsi: bool = True
    bollinger_bands: bool = True
    macd: bool = True
    ichimoku: bool = True
    stochastic_momentum_index: bool = True
    stochastic_oscillator: bool = True
    williamR: bool = True
    fisherT: bool = True
    average_true_range: bool = True
    obv: bool = True
    zscore: bool = True
    entropy: bool = True
    candle_stick: bool = True
    relative_price: bool = True
    onehot_vector_size: int = 512

    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = True
    datetime_features: int = 55

    ## Training
    run_name: str = "aapl_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "aapl"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "transformer_rolling"

In [39]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['aapl']

['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 'CDL_MORNINGSTAR', 'CDL_3BLACKCROWS', 'CDL_3LINESTRIKE', 'Open/Close', 'High/Close', 'Low/Close', 'Close_Change', 'golden_crossover', 'death_crossover', 'rsi_oversold', 'rsi_overbought', 'bbands_high_volatility', 'bbands_low_volatility', 'bbands_buy_signal', 'bbands_sell_signal', 'macd_golden_crossover', 'macd_death_crossover', 'macd_buy_signal_histogram', 'macd_sell_signal_histogram', 'macd_buy_signal_zero_cross', 'macd_sell_signal_zero_cross', 'ichimoku_buy_signal', 'ichimoku_sell_signal', 'smi_buy_signal', 'smi_sell_signal', 'stoch_osc_buy_signal', 'stoch_osc_sell_signal', 'williamR_buy_signal', 'williamR_sell_signal', 'fisherT_buy_signal', 'fisherT_sell_signal', 'atr_buy_signal', 'atr_sell_signal', 'obv_buy_signal', 'obv_sell_signal', 'zscore_buy_signal', 'zscore_sell_signal', 'entropy_buy_signal', 'entropy_sell_signal']
Dataset Start Year: 1990 | End Year: 2023
['CDL_EVENINGSTAR', 'CDL_3WHITESOLDIERS', 

In [40]:
trainer.run()

Training model: aapl_run1



  0%|          | 0/21 [00:00<?, ?it/s]

Year: 2000 | Validation Accuracy: 46.83% | Only Long Accuracy: 46.83% | Highest Validation Accuracy: 0.00%
Year: 2001 | Validation Accuracy: 47.58% | Only Long Accuracy: 46.37% | Highest Validation Accuracy: 0.00%
Training Years: 1990~1999 | Training Accuracy: 51.23% | Training OL Accuracy: 47.47%
Year: 2000~2001 Validation Accuracy: 47.20% | Only Long Accuracy: 46.60% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 46.03% | Only Long Accuracy: 46.03%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/transformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Year: 2000 | Validation Accuracy: 54.76% | Only Long Accuracy: 46.83% | Highest Validation Accuracy: 47.20%
Year: 2001 | Validation Accuracy: 59.27% | Only Long Accuracy: 46.37% | Highest Validation Accuracy: 47.20%
Training Years: 1990~1999 | Training Accuracy: 52.73% | Training OL Accuracy: 47

  5%|▍         | 1/21 [05:39<1:53:12, 339.63s/it]

Training Years: 1990~1999 | Training Accuracy: 99.68% | Training OL Accuracy: 47.47%
Year: 2000~2001 Validation Accuracy: 69.40% | Only Long Accuracy: 46.60% | Highest Validation Accuracy: 74.60%
Year: 2002 | Test Accuracy: 66.67% | Only Long Accuracy: 46.03%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 74.60% | Highest Test Accuracy in 2002: 66.67%
***********************************************************
=============================New Training Set====================================


...

Training Years: 2009~2018 | Training Accuracy: 99.64% | Training OL Accuracy: 47.62%
Year: 2019~2020 Validation Accuracy: 68.12% | Only Long Accuracy: 47.33% | Highest Validation Accuracy: 73.47%
Year: 2021 | Test Accuracy: 67.86% | Only Long Accuracy: 49.60%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 73.47% | Highest Test Accuracy in 2021: 67.86%
***********************************************************
=============================New Training Set====================================


<ipython-input-37-5a0e3e4143c0>:262: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Model loaded from /content/drive/MyDrive/code/fintransformer/models/transformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 73.47% | Test Accuracy: 68.25%
Year: 2020 | Validation Accuracy: 71.54% | Only Long Accuracy: 47.83% | Highest Validation Accuracy: 0.00%
Year: 2021 | Validation Accuracy: 69.44% | Only Long Accuracy: 49.60% | Highest Validation Accuracy: 0.00%
Training Years: 2010~2019 | Training Accuracy: 95.95% | Training OL Accuracy: 47.77%
Year: 2020~2021 Validation Accuracy: 70.50% | Only Long Accuracy: 48.71% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 68.92% | Only Long Accuracy: 48.21%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/transformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Year: 2020 | Validation Accuracy: 73.52% | Only Long 

100%|██████████| 21/21 [1:47:16<00:00, 306.49s/it]

Training Years: 2010~2019 | Training Accuracy: 99.40% | Training OL Accuracy: 47.77%
Year: 2020~2021 Validation Accuracy: 70.30% | Only Long Accuracy: 48.71% | Highest Validation Accuracy: 72.48%
Year: 2022 | Test Accuracy: 67.73% | Only Long Accuracy: 48.21%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 72.48% | Highest Test Accuracy in 2022: 67.73%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2000~2001 : 74.60% | Test Accuracy in 2002: 72.22%
Validation Accuracy in 2001~2002 : 72.80% | Test Accuracy in 2003: 71.03%
Validation Accuracy in 2002~2003 : 70.44% | Test Accuracy in 2004: 65.08%
Validation Accuracy in 2003~2004 : 70.83% | Test Accuracy in 2005: 64.29%
Validation Accuracy in 2004~2005 : 67.06% | Test Accuracy in 2006: 69.32%
Validation Accuracy in 2005~2

In [ ]:
from google.colab import runtime
runtime.unassign()